<a href="https://colab.research.google.com/github/coopercunliffe/springboard/blob/master/tweet_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.set_option('display.max_colwidth', 72)

# The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.


def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
            'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + '_' + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)


In [0]:
df09 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2009.json')
df10 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2010.json')
df11 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2011.json')
df12 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2012.json')
df13 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2013.json')
df14 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2014.json')
df15 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2015.json')
df16 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2016.json')
df17 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2017.json')
df18 = pd.read_json('/content/drive/My Drive/twitter data/condensed_2018.json')
df19 = pd.read_json('/content/drive/My Drive/twitter data/2019.json')
df20 = pd.read_json('/content/drive/My Drive/twitter data/2020.json')

In [0]:
df19.columns

Index(['source', 'id_str', 'text', 'created_at', 'retweet_count',
       'in_reply_to_user_id_str', 'favorite_count', 'is_retweet'],
      dtype='object')

In [0]:
# df19 is corrupted, many retweets incorrectly labelled False. Here we fix that

df19['is_retweet'] = df19.apply(lambda row: True 
                                 if row['text'].startswith('RT @')
                                 else False, axis=1)

In [0]:
years = [df09, df10, df11, df12, df13, df14, 
         df15, df16, df17, df18, df19, df20]

df = pd.concat(years)
df = df.sort_values('created_at').reset_index(drop=True)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48151 entries, 0 to 48150
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   source                   48151 non-null  object             
 1   id_str                   48151 non-null  int64              
 2   text                     48151 non-null  object             
 3   created_at               48151 non-null  datetime64[ns, UTC]
 4   retweet_count            48151 non-null  int64              
 5   in_reply_to_user_id_str  3301 non-null   float64            
 6   favorite_count           48151 non-null  int64              
 7   is_retweet               48151 non-null  bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(3), object(2)
memory usage: 2.6+ MB


In [0]:
df.iloc[0]

source                                                                          Twitter Web Client
id_str                                                                                  1698308935
text                       Be sure to tune in and watch Donald Trump on Late Night with David L...
created_at                                                               2009-05-04 18:54:25+00:00
retweet_count                                                                                  253
in_reply_to_user_id_str                                                                        NaN
favorite_count                                                                                 202
is_retweet                                                                                   False
Name: 0, dtype: object

In [0]:


df['created_date'] = [d.date() for d in df['created_at']]
df['created_time'] = [d.time() for d in df['created_at']]

# takes a retweet as a string and removes the 'RT @name: '
def remove_rt_signature(tweet):
    split_tweet = tweet.split(':', 1)
    return split_tweet[1][1:]

# make a new column of all the tweets with the retweets cleaned up
# df['text_cleaned'] = df.apply(lambda row: row['text'] if row['is_retweet'] == False else remove_rt_signature(row['text']), axis=1)



# make a column that indicates the hour of tweet

df['created_hour'] = pd.to_datetime(df['created_time'].astype(str)).dt.hour


# add a column for hours changed from GMT to EST

def gmt_to_est(gmt):
  est = (gmt - 4) % 24
  return est

df['created_hour_est'] = df.apply(lambda row: gmt_to_est(row['created_hour']), axis=1)


# make a column that classifies tweet as night or day tweet (2:00 - 10:00 GMT = 22:00 - 6:00 EST)

# df['is_night_tweet'] = df['created_hour'].isin([2, 3, 4, 5, 6, 7, 8, 9, 10])


# make a column that classifies tweets as before and after becoming president


df['is_presidential'] = df.apply(lambda row: True 
                                 if row['created_date'] >= dt.date(2017,1,20)
                                 else False, axis=1)      # (2017,01,20)


# add a validation column to indicate the validation data, eventually set most recent 10-20% to True

df['is_valid'] = False


# add the datetime catagoricals

add_datepart(df, 'created_at', drop=False)


# add a column that describes the days of the week in strings instead of integers

day_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df['created_at_Dayofweek_str'] = [day_list[i] for i in df['created_at_Dayofweek']]

In [0]:
df.shape

(48151, 28)

In [0]:
df.iloc[-1]

source                                                                              Twitter for iPhone
id_str                                                                             1259843806049992704
text                           Dems are trying to steal the Mike Garcia Congressional Race in Calif...
created_at                                                                   2020-05-11 13:52:24+00:00
retweet_count                                                                                     1557
in_reply_to_user_id_str                                                                            NaN
favorite_count                                                                                    4225
is_retweet                                                                                       False
created_date                                                                                2020-05-11
created_time                                                             

In [0]:
df.to_csv('/content/drive/My Drive/twitter data/tweet_df.csv')

In [0]:
tweets_by_day = pd.DataFrame(df['created_date'].value_counts().sort_index())
tweets_by_day = tweets_by_day.rename(columns={'created_date':'total number of tweets'})
tweets_by_day['retweets'] = [df[(df['created_date'] == i) & (df['is_retweet'] == True)].shape[0] for i in tweets_by_day.index]
print(tweets_by_day.iloc[-220:-170])

# fixed

            total number of tweets  retweets
2019-10-05                      45        15
2019-10-06                      44        19
2019-10-07                      26         6
2019-10-08                      51        29
2019-10-09                      49        13
2019-10-10                      24         4
2019-10-11                      52        31
2019-10-12                      28        10
2019-10-13                      24         4
2019-10-14                      34         3
2019-10-15                      15         0
2019-10-16                      25         6
2019-10-17                      32        14
2019-10-18                      22         5
2019-10-19                      55        35
2019-10-20                      22         5
2019-10-21                      18         6
2019-10-22                      41        27
2019-10-23                      47        25
2019-10-24                      15         6
2019-10-25                      42        24
2019-10-26